In [28]:
%%html
<style>
table {float:left}
</style>

In [1]:
import os
import xml.etree.ElementTree as et

#### Functions

In [2]:
import copy

def calc_precision(tp, fp):
    return tp/(tp + fp)

def calc_recall(tp, fn):
    return tp/(tp + fn)

def calc_fscore(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

def evaluate(gold_truth_labels, predictions):
    # Counts of true positives, false positives & false negatives
    tp, fp, fn = 0, 0, 0
    
    # List with false positives and false negatives
    fps, fns = [], []
    
    for gold, pred in zip(gold_truth_labels, predictions):
        
        tp_tmp, fp_tmp, fn_tmp, fns_temp, fps_temp  = evaluate_one_article(gold, pred)
        
        tp += tp_tmp
        fp += fp_tmp
        fn += fn_tmp
        
        fns.extend(fns_temp)
        fps.extend(fps_temp) 
        
    precision = calc_precision(tp, fp)
    recall = calc_recall(tp, fn)
    f_score = calc_fscore(precision, recall)    
    
    print(f'fp: {fp} | tp: {tp} | fn: {fn}')
    print(f'precision: {precision} | recall: {recall} | f-score: {f_score}')
    
    return fps, fns  
    

def evaluate_one_article(gold_truth, prediction):
    
    gold = gold_truth['entities'].copy()
    pred = prediction['entities'].copy()
    
    # Counts of true positives, false positives & false negatives
    tp, fp, fn = 0, 0, 0
    
    # List with false positives and false negatives
    fps, fns = [], []
    
    
    i = 0
    
    while len(gold) > 0 and len(pred) > 0:
        i += 1

        # Check if the first two elements are the same
        if gold[0] == pred[0]:
            tp += 1
            gold.pop(0)
            pred.pop(0)
        
        else:
            # Grab the first appearing element
            element, source = (gold[0], 'gold') if gold[0]['start_pos'] < pred[0]['start_pos'] else (pred[0], 'pred')
            
            # Remove the element first appearing element
            if source == 'gold':
                fn += 1
                fns.append(element['text'])
                gold.remove(element)
            elif source == 'pred':
                fp += 1
                fps.append(element['text'])
                pred.remove(element)
    
    if len(gold) > 0:
        fn += 1
    elif len(pred) > 0:
        fp += 1
        
    return tp, fp, fn, fns, fps       

def run_flair(text):

    # make a sentence
    sentence = Sentence(text)

    # run NER over sentence
    tagger.predict(sentence)
    
    for entity in sentence.to_dict(tag_type='ner')['entities']:
        print(entity)

## TR-News

In [60]:
# Get file path LGL dataset
file_path = '../../data/TR-News/TR-News.xml'

# Load the data
tree = et.parse(file_path)
root = tree.getroot()

# Grab example title
example = root[0][0].text
example

'Policeman shot dead after assassinating Russian ambassador to Turkey, shouting ‘Don’t forget Aleppo!’\n        '

#### Ground truth labels

In [61]:
all_ground_truth = []

for article in root:
    
    gold_truth = {'text': article.find('text').text,
                  'entities': sorted([{'text': top.find('phrase').text,
                                'start_pos': int(top.find('start').text),
                                'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
#                                  if top.find('gaztag/lat') != None and top.find('gaztag/lon') != None
                                     ], key=lambda k: k['start_pos'])}
    
    
    all_ground_truth.append(gold_truth)


#### List of predictions

In [62]:
from tqdm import tqdm

In [63]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the NER tagger
tagger = SequenceTagger.load('ner')

predictions = []

for article in tqdm(all_ground_truth):
    
    text = article['text']
    
    # make a sentence
    sentence = Sentence(text)
    
    # run NER over sentence
    tagger.predict(sentence)
    
    pred = sentence.to_dict(tag_type='ner')
    pred['entities'] = [entity for entity in pred['entities'] if entity['labels'][0].value == 'LOC']
    [entity.pop('labels') for entity in pred['entities']]
    pred.pop('labels')
    
    predictions.append(pred)
    
    

2021-01-21 13:52:01,389 loading file C:\Users\Bernard\.flair\models\en-ner-conll03-v0.4.pt


100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [01:30<00:00,  1.31it/s]


#### Results TR-News & Comparison

In [45]:
# only toponyms w/ lat/long
fps, fns = evaluate(all_ground_truth, predictions)

fp: 255 | tp: 867 | fn: 380
precision: 0.7727272727272727 | recall: 0.6952686447473937 | f-score: 0.7319544111439427


In [64]:
# all toponyms
fps, fns = evaluate(all_ground_truth, predictions)

fp: 222 | tp: 903 | fn: 388
precision: 0.8026666666666666 | recall: 0.6994577846630519 | f-score: 0.7475165562913907


In [65]:
fps

['White House',
 'Islamic State',
 'White House',
 'Islamic State',
 "Faulkners'",
 'Interstate 64',
 'South Regional Jail',
 'Granville',
 'Born',
 'Amazon',
 'Rose Garden',
 'Texas A&M University',
 'Texas A&M',
 'Ronald Reagan Building',
 'Islamic State',
 'Cumberland Farms',
 'South         Main St.Bingham',
 'Calgary-North West',
 'Kremlin',
 'Kremlin',
 'Downtown Eastside',
 'London City',
 'London City',
 'Central             Anatolia',
 'Dolby Theatre',
 'Etobicoke',
 'Sage House',
 'Churchill',
 'Churchill',
 "Children's Hospital",
 'Manitoba',
 'Winnipeg',
 'US',
 'Phoenix',
 'Phoenix',
 'Gatineau',
 'Que.',
 'Ottawa',
 'Karak',
 'Crusader',
 'Paris             Town Hall',
 'La Liga',
 'Petit Cambodge',
 'St. Peter',
 'St. Paul',
 'Islamic Sharia',
 'Giza',
 'Saint Peter',
 'Saint Paul Coptic Orthodox Church',
 'Mathari National Teaching and Referral Hospital',
 'B.C',
 'Westminster House',
 'Ottawa',
 'Ottawa',
 'Ottawa',
 'Ottawa',
 'Ottawa',
 'Trillium Health Centre',
 'Mi

In [67]:
fns

['Turkish',
 'Turkish',
 'Syrian',
 'Syrian',
 'Turkish',
 'Kurdish',
 'Turkish',
 'Russian',
 'Syrian',
 'Russian',
 'Russia',
 'Turkey',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Turkish',
 'Turkish',
 'Russian',
 'Russian',
 'Turkish',
 'Russian',
 'Russian',
 'Russian',
 'Syrian',
 'Granville County',
 'West Virginia',
 'Texas',
 'Texas',
 'Texas',
 'U.S.',
 'Xavier University',
 'British',
 'European',
 'U.S.',
 'Ohio',
 'U.S.',
 'New York',
 'Michigan',
 'Cuban',
 'BANTAM',
 'Bantam',
 'New Milford',
 'Calgary',
 'Russian',
 'Russian',
 'Russian',
 'Russian',
 'London',
 'London',
 'Anatolia',
 'German',
 'Iraqi',
 'Iraqi',
 'German',
 'Spanish',
 'Venice',
 'Canadian',
 'Cannes',
 'Spanish',
 'France',
 'French',
 'French',
 'Spanish',
 'Berlin',
 'German',
 'European',
 'American',
 'Chinese',
 'China',
 'American',
 'Macdonald-Cartier International Airport',
 'Canada',
 'Canada',
 'Man.',
 'Churchill',
 'Churchill',
 'Churchill',
 'Manitoba',
 'Winnipeg',
 

## LGL

In [68]:
# Get file path LGL dataset
file_path = '../../data/LGL/LGL.xml'

# Load the data
tree = et.parse(file_path)
root = tree.getroot()

#### Ground truth labels

In [69]:
all_ground_truth = []

for article in root:
    
    gold_truth = {'text': article.find('text').text,
                  'entities': sorted([{'text': top.find('phrase').text,
                                'start_pos': int(top.find('start').text),
                                'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
#                                               if top.find('gaztag/lat') != None and top.find('gaztag/lon') != None
                                     ], key=lambda k: k['start_pos'])}
    
    
    all_ground_truth.append(gold_truth)



#### List of predictions

In [70]:
from tqdm import tqdm

In [71]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the NER tagger
tagger = SequenceTagger.load('ner')

predictions = []

for article in tqdm(all_ground_truth):
    
    text = article['text']
    
    # make a sentence
    sentence = Sentence(text)
    
    # run NER over sentence
    tagger.predict(sentence)
    
    pred = sentence.to_dict(tag_type='ner')
    pred['entities'] = [entity for entity in pred['entities'] if entity['labels'][0].value == 'LOC']
    [entity.pop('labels') for entity in pred['entities']]
    pred.pop('labels')
    
    predictions.append(pred)
    
    

2021-01-21 14:31:47,420 loading file C:\Users\Bernard\.flair\models\en-ner-conll03-v0.4.pt


100%|████████████████████████████████████████████████████████████████████████████████| 588/588 [07:21<00:00,  1.33it/s]


#### Results LGL & comparison

In [15]:
# filter toponyms (only w/ lat & long)
fps, fns = evaluate(all_ground_truth, predictions)

fp: 1465 | tp: 2848 | fn: 1514
precision: 0.660329237189891 | recall: 0.6529115084823476 | f-score: 0.6565994236311239


In [72]:
# all toponyms
fps, fns = evaluate(all_ground_truth, predictions)

fp: 1049 | tp: 3370 | fn: 1614
precision: 0.7626159764652637 | recall: 0.6761637239165329 | f-score: 0.716792513027757


In [74]:
fns

['Rapides Parish',
 'Avoyelles',
 'Cottonport',
 'Alexandria',
 'Pointe Coupee',
 'MANSFIELD',
 'Mansfield',
 'Shreveport',
 'Mansfield',
 'Mansfield',
 'DeSoto Parish',
 'Cook',
 'Minneapolis',
 'Minnesota',
 'Minnesota',
 'Marshall',
 'Chisholm',
 'Otter Tail County',
 'Otter Tail/Grant',
 'Grant/Wilkin',
 'County Road 43',
 'Douglas County',
 'Minnesota',
 'Minnesota',
 'Alexandria',
 'Fargo',
 'Moorhead',
 'Fargo',
 'Minnesota',
 'Fargo',
 'Fargo',
 'Fargo',
 'Fargo',
 'Douglas County',
 'Nelson',
 'Douglas County',
 'Alexandria',
 'Alexandria',
 'Alexandria',
 'Washington',
 'D.C.',
 'D.C.',
 'Alexandria',
 'Virginia',
 'Sudanese',
 'Sudan',
 'Chinese',
 'Africa',
 'African',
 'Darfur',
 'Egyptian',
 'Egyptian',
 'Sudan',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Egyptian',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Sudanese',
 'Egyptian',
 'Sudanese',
 'Sudanese',
 'Alexandria',
 'Sudanese',
 'Sri Lankan',
 'Sri Lankan',
 'Sinhalese',
 'Sinhale

In [75]:
fps

['Kelleyland',
 'Orchard St.',
 'Cottonport Fire Station',
 'Warden',
 'St. James Youth Detention Center',
 'DFL-Minneapolis',
 'Minnesotaâ',
 'Capitol',
 'R-Elbow Lake',
 'Otter Tail County Road',
 'Highway 59',
 'Highway 59',
 'Otter Tail',
 'Grant',
 'Highway 55',
 'Highway 55',
 'Grant',
 'Wilkin',
 'County Road',
 'Douglas County Road',
 'Highway 9',
 'Highway 12',
 'Highway 75',
 'Room 113',
 'Highway 29',
 'Fargo-Moorhead',
 'Hesco',
 'Barb Grothâ',
 'Fargodome',
 'Sandbag Central',
 'Re',
 'Max',
 'Dodge Caravan',
 'Douglas County Hospital',
 'St. Cloud Hospital',
 'Cora Kelly Elementary School',
 'Minnie Howard',
 'Durant Center',
 'Hill',
 'Northwest D.C.',
 'East End',
 'Southern Sudan',
 'Sub-Saharan Africa',
 'Egypt-US',
 'Gulf',
 'Gulf',
 'US',
 'US',
 'US',
 'Gulf',
 'US',
 'Gulf',
 'US',
 "St. John's Lutheran School",
 'Sheldon Peck Homestead',
 'Crate & Barrel',
 'Woodfield Shopping Center',
 'Hudson',
 'Decatur McLean County',
 'O.D',
 'Houston',
 'City Hall',
 'E',
 

## GeoWebNews

In [77]:
# Get file path LGL dataset
file_path = '../../data/GeoWebNews/GeoWebNews.xml'

# Load the data
tree = et.parse(file_path)
root = tree.getroot()

#### Ground truth labels

In [78]:
all_ground_truth = []

for article in root:
    
    gold_truth = {'text': article.find('text').text,
                  'entities': sorted([{'text': top.find('extractedName').text,
                                'start_pos': int(top.find('start').text),
                                'end_pos': int(top.find('end').text)} for top in article.findall('toponyms/toponym')
                                              if top.find('latitude') != None and top.find('longitude') != None
                                     ], key=lambda k: k['start_pos'])}
    
    
    all_ground_truth.append(gold_truth)



#### List of predictions

In [79]:
from tqdm import tqdm

In [80]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the NER tagger
tagger = SequenceTagger.load('ner')

predictions = []

for article in tqdm(all_ground_truth):
    
    text = article['text']
    
    # make a sentence
    sentence = Sentence(text)
    
    # run NER over sentence
    tagger.predict(sentence)
    
    pred = sentence.to_dict(tag_type='ner')
    pred['entities'] = [entity for entity in pred['entities'] if entity['labels'][0].value == 'LOC']
    [entity.pop('labels') for entity in pred['entities']]
    pred.pop('labels')
    
    predictions.append(pred)
    
    

2021-01-21 17:14:59,993 loading file C:\Users\Bernard\.flair\models\en-ner-conll03-v0.4.pt


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [03:10<00:00,  1.05it/s]


#### Results GeoWebNews

In [81]:
# only toponyms with long / lat info
fps, fns = evaluate(all_ground_truth, predictions)

fp: 183 | tp: 1669 | fn: 851
precision: 0.9011879049676026 | recall: 0.6623015873015873 | f-score: 0.763494967978042


In [25]:
# all toponyms --> fn much higher because many annotated toponyms aren't locations (not sure why this is)
fps, fns = evaluate(all_ground_truth, predictions)

fp: 165 | tp: 1693 | fn: 4089
precision: 0.9111948331539289 | recall: 0.29280525769629884 | f-score: 0.44319371727748685


In [82]:
fns

['Louisiana',
 'French',
 'French Quarter',
 'German',
 'Irish',
 'Franklin',
 'Methodist church',
 '2231 Royal',
 'Greek',
 'New Orleans Railways and Light Company Claiborne Power House',
 'French',
 'Elysian Fields',
 'Carnegie Library',
 'Kurdish',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Syrian',
 'Turkish',
 'Syrian',
 'Kurdish',
 'Turkish',
 'Britain',
 'Syrian',
 'Kurdish',
 'Turkish',
 'Syrian',
 'Stonewall Golf Club',
 'Virginia Gateway',
 'Old Town Manassas',
 'Vienna/Fairfax-GMU Metro',
 'Washington',
 'Washington',
 'Florida Atlantic University',
 'California Polytechnic State University',
 'California State University',
 'University of California',
 'UCLA',
 'Washington',
 'African',
 'African',
 'African',
 'African',
 'African',
 'African',
 'African',
 'Russian',
 'Russian',
 'Russian Higher School of Economics',
 'Connecticut',
 'Mississippi',
 'Louisville',
 'Mississippi',
 'American',
 'Louisville',
 'Mississippi',
 'Louisville',
 'Nigerian',
 'Nigerian',
 'Muscat',
 'Om

In [21]:
fps

['D’Aunoy',
 'Royal',
 'Desire',
 'New Orleans Railways',
 'Light Company Claiborne Power House',
 'Dubreuil',
 'Smoky Mary',
 'AP',
 'Jamison',
 'Norbury',
 'Granville',
 'Somerville',
 'DuraDeck',
 'Jamison',
 'Norbury',
 'Norbury',
 'Wegmans',
 'Bed Bath & Beyond',
 'Walmart',
 'Kohl',
 'Old Town',
 'Manassas',
 'Vienna',
 'Fairfax-GMU Metro',
 'Orange',
 'Turtle Creek Cir.',
 'Va',
 'Kremlin',
 'Mississippi State',
 'Mississippi State',
 'Victoria Vivians’',
 'Ogunbowale',
 'Mississippi State',
 'Mississippi State',
 'White House',
 'White House',
 'Rappler',
 'Story Course',
 'Ayton',
 'North America',
 'Gate 10',
 'Treasure Beach',
 'Animal Kingdom',
 'Islamic State',
 'West',
 'Kremlin',
 'Asia-Pacific',
 'Tianjin No. 2 Detention Centre',
 'Seasons',
 'Vanni',
 'Bakgatla-ba-Kgafela',
 'Bakgatla',
 'Washington',
 'D.C.',
 'D.C',
 '22nd St. North',
 'Eurostar',
 'HH',
 'Eurostar',
 'Amsterdam-London',
 'Magtulungan',
 'Read More',
 'Umuanunu',
 'Benue',
 'Peshawar',
 'AP',
 'Monta